In [5]:
! pip install datasets

In [6]:
from datasets import load_dataset

# Load the dataset directly from Hugging Face Hub
dataset = load_dataset("google/Synthetic-Persona-Chat")

# Convert to DataFrames if needed
df_train = dataset["train"].to_pandas()
df_val = dataset["validation"].to_pandas()
df_test = dataset["test"].to_pandas()

print(df_train.head())


README.md:   0%|          | 0.00/3.11k [00:00<?, ?B/s]

c:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\datasets--google--Synthetic-Persona-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Synthetic-Persona-Chat_train.csv:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

Synthetic-Persona-Chat_valid.csv:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Synthetic-Persona-Chat_test.csv:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8938 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/968 [00:00<?, ? examples/s]

                                     user 1 personas  \
0  I am 32.\nI do not want a job.\nI play video g...   
1  I am 32.\nI play video games all day.\nI still...   
2  I am 32.\nI play video games all day.\nI still...   
3  I write.\nI work at mcdonald s.\nI watch youtu...   
4  I am bald.\nI like to swim.\nMy favorite drink...   

                                     user 2 personas  \
0  My favorite drink is iced coffee.\nI have a bl...   
1  I have a ford f150.\nI like ford cars.\nMy tru...   
2  I can recite the movie young frankenstein word...   
3  I want to move.\nI don t like feeling controll...   
4  My favorite store is american eagle.\nI enjoy ...   

                         Best Generated Conversation  
0  User 1: Hi! I'm [user 1's name].\nUser 2: Hi [...  
1  User 1: Hey, how's it going?\nUser 2: Good, I'...  
2  User 1: Hi, my name is John. What's your name?...  
3  User 1: Hi!\nUser 2: Hey!\nUser 1: What's up?\...  
4  User 1: Hello!\nUser 2: Hi!\nUser 1: What do y..

In [ ]:
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("google/Synthetic-Persona-Chat")

# Use the train split for quick testing
data = dataset["train"]

# Show an example conversation
print("Example row:")
print(data[0])


Example row:
{'user 1 personas': 'I am 32.\nI do not want a job.\nI play video games all day.\nI still live at home with my parents.', 'user 2 personas': 'My favorite drink is iced coffee.\nI have a black belt in karate.\nI m in a jazz band and play the saxophone.\nI vacation along lake michigan every summer.', 'Best Generated Conversation': "User 1: Hi! I'm [user 1's name].\nUser 2: Hi [user 1's name], I'm [user 2's name].\nUser 1: What do you do for fun?\nUser 2: I like to play video games, go to the beach, and read.\nUser 1: I like to play video games too! I'm not much of a reader, though.\nUser 2: What video games do you like to play?\nUser 1: I like to play a lot of different games, but I'm really into competitive online games right now.\nUser 2: I'm not really into competitive games, I like to play more relaxing games.\nUser 1: That's cool. What kind of relaxing games do you like to play?\nUser 2: I like to play puzzle games, simulation games, and story-based games.\nUser 1: I've

In [ ]:
class MemoryBot:
    def __init__(self):
        self.memory = {}

    def tell(self, statement):
        # Basic parsing like: "my money is in the wallet"
        if " is in " in statement:
            parts = statement.split(" is in ")
            item = parts[0].strip().lower()
            location = parts[1].strip().lower()
            self.memory[item] = location
            return "Alright, I will remember that."
        return "Okay."

    def ask(self, question):
        # Basic question parsing
        for item in self.memory:
            if item in question.lower():
                return f"You put your {item} in the {self.memory[item]}."
        return "I'm not sure."

# ----------------------------
# Test it
bot = MemoryBot()

# Tell the bot something
print("You: my money is in the wallet")
print("Bot:", bot.tell("my money is in the wallet"))

# Ask the bot a question
print("\nYou: where did I put my money?")
print("Bot:", bot.ask("where did I put my money?"))


You: my money is in the wallet
Bot: Alright, I will remember that.

You: where did I put my money?
Bot: You put your my money in the the wallet.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Load the dataset
dataset = load_dataset("google/Synthetic-Persona-Chat")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

# Combine persona + history into one input string
def preprocess(example):
    input_text = f"{example['persona']} [SEP] {' '.join(example['history'])}"
    target_text = example['reply']
    full_text = input_text + " " + target_text
    tokens = tokenizer(full_text, truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized = dataset["train"].map(preprocess, remove_columns=dataset["train"].column_names)

# Model
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# Training setup
args = TrainingArguments(
    output_dir="./dialogpt-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

trainer.train()


In [ ]:
# Load your trained model
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained("./dialogpt-finetuned")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

# Chat loop
chat_history_ids = None

print("🤖 Trained Chatbot is ready. Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.92,
        top_k=50
    )

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print("Bot:", response)


## Build model

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import os

# 1. Load dataset
dataset = load_dataset("google/Synthetic-Persona-Chat")

# 2. Split dataset into train and eval (80/20 split)
split_dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 3. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 4. Preprocessing function
def preprocess(example):
    # Get the conversation turns
    conversation = example["Best Generated Conversation"]
    
    # Join all turns except the last one as context
    context = "\n".join(conversation[:-1])
    # The last turn is the response we want to predict
    response = conversation[-1]
    
    # Tokenize context and response
    model_inputs = tokenizer(
        context,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    
    # Tokenize response for labels
    labels = tokenizer(
        text_target=response,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 5. Tokenize datasets
tokenized_train = train_dataset.map(
    preprocess,
    batched=False,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    preprocess,
    batched=False,
    remove_columns=eval_dataset.column_names
)

# 6. Training arguments - with better saving configuration
training_args = TrainingArguments(
    output_dir="./dialoGPT-finetuned",  # Changed to more specific directory
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    save_safetensors=True,  # Save in safe format
    load_best_model_at_end=True,  # Will load best model at end
    metric_for_best_model="eval_loss",  # Use eval loss to determine best
    greater_is_better=False,  # Lower eval loss is better
    push_to_hub=False,
    report_to="wandb",
    run_name="dialoGPT-persona-chat"
)

# 7. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)

# 8. Train and save
trainer.train()

# 9. Save the final model and tokenizer
final_output_dir = "./dialoGPT-finetuned-final"
os.makedirs(final_output_dir, exist_ok=True)

trainer.save_model(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

print(f"Model saved to {final_output_dir}")

In [ ]:
print("Hello")